# Generating DeepLoc Train/Valid/Test Data

In this notebook, we parse through the original DeepLoc dataset to generate the train/test splits as well as a validation set from the test set. You can specify the max sequence length for your output data, which in our case we used 6000 due to memory issues on EC2 when training on a GPU. 

Paper: https://academic.oup.com/bioinformatics/article/33/21/3387/3931857

Dataset: http://www.cbs.dtu.dk/services/DeepLoc-1.0/deeploc_data.fasta

Additionally, we remove the Cytoplasm/Nucleus class as done in the original DeepLoc paper to mimic their data cleaning process.

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import json

np.random.seed(42)

First, we will download the deeploc dataset to the /data folder within the TAPE project

In [26]:
!curl -o ../data/deeploc_data.fasta http://www.cbs.dtu.dk/services/DeepLoc-1.0/deeploc_data.fasta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7470k  100 7470k    0     0  1254k      0  0:00:05  0:00:05 --:--:-- 1782k:--  0:05:36 22703


Then, we will define a max sequence length and iterate through the DeepLoc dataset to output a filtered fasta file. Next, we generate train/test DataFrames in order to write out the train/test/valid split fasta files.

Some sequences in the DeepLoc dataset have a Cytoplasm-Nucleus label for the subcellular location. Per the DeepLoc dataset, we are excluding these labels since they do not clearly fall into the Cytoplasm or Nucleus classes.

In [37]:
# Lists to store train and test sets. The original dataset indicates whether 
# each sequence belongs in the train or test set in the record description
train_list = []
test_list = []

# Create a mapping from protein sequence ID to the q10 subcellular location labels
# and the q2 membrane bound vs water soluble labels
seq_id_to_q10 = {}
seq_id_to_q2 = {}

# Define max sequence length to filter data. Larger sequences take more memory to encode.
MAX_SEQ_LENGTH = 6000

with open(f"../data/deeploc_data_{MAX_SEQ_LENGTH}.fasta", 'w') as handle:
    # Iterate through the original dataset to parse for ID, sequence, class label, and train/test flag
    for record in SeqIO.parse("../data/deeploc_data.fasta", "fasta"):
        description = record.description
        seq = record.seq
        desc_split = description.split(" ")
        ID = desc_split[0]
        label = desc_split[1]
        q2_label = label[-1] # q2 membrane bound vs water soluble label
        q10_label = label[:len(label)-2] # q10 subcellular location label
        
        # Ignore ambiguous cytoplasm-nucleus labels and sequences that are too long to embed
        if q10_label == "Cytoplasm-Nucleus" or len(seq) > MAX_SEQ_LENGTH:
            continue
        
        seq_id_to_q2[ID] = q2_label
        seq_id_to_q10[ID] = q10_label
        # Sequences in test set has an additional field in the description
        if len(desc_split) == 3:
            test_list.append((ID, q10_label, seq))
        else:
            train_list.append((ID, q10_label, seq))
        SeqIO.write(record, handle, "fasta")

# Create pd DataFrames for the train and test sequences
train_df = pd.DataFrame(train_list)
test_df = pd.DataFrame(test_list)

# Generate json label files for visualizating the embeddings with PCA
with open('../data/deeploc_labels_q2.json', 'w') as handle:
    handle.write(json.dumps(seq_id_to_q2))

with open('../data/deeploc_labels_q10.json', 'w') as handle:
    handle.write(json.dumps(seq_id_to_q10))

In [38]:
print(train_df.shape)
print(test_df.shape)

(11083, 3)
(2773, 3)


Inspect the DataFrame
* Column 0 is a unique identifier for each sequence (sequence ID)
* Column 1 shows the labels for the q10 and q2 tasks, separated by a dash -
* Column 2 is a tuple of the amino acids in each sequence

In [39]:
train_df.head()

,0,1,2
0,Q5I0E9,Cell.membrane,"(M, E, V, L, E, E, P, A, P, G, P, G, G, A, D, ..."
1,P63033,Cell.membrane,"(M, M, K, T, L, S, S, G, N, C, T, L, N, V, P, ..."
2,Q9NR71,Cell.membrane,"(M, A, K, R, T, F, S, N, L, E, T, F, L, I, F, ..."
3,Q86XT9,Cell.membrane,"(M, G, N, C, Q, A, G, H, N, L, H, L, C, L, A, ..."
4,A2CI98,Cell.membrane,"(M, D, P, S, K, Q, G, T, L, N, R, V, E, N, S, ..."


Looking at the distribution of the output classes for subcellular location (q10), you can see that the dataset is imbalanced. Almost 50% of sequences are either in the Nucleus or Cytoplasm classes.

In [40]:
train_df[1].value_counts()

Nucleus                  3235
Cytoplasm                2033
Extracellular            1579
Mitochondrion            1208
Cell.membrane            1067
Endoplasmic.reticulum     689
Plastid                   605
Golgi.apparatus           286
Lysosome/Vacuole          257
Peroxisome                124
Name: 1, dtype: int64

In [41]:
test_df[1].value_counts()

Nucleus                  808
Cytoplasm                508
Extracellular            393
Mitochondrion            302
Cell.membrane            273
Endoplasmic.reticulum    173
Plastid                  152
Golgi.apparatus           70
Lysosome/Vacuole          64
Peroxisome                30
Name: 1, dtype: int64

Create map from subcellular location label to a numerical ID to use for stratifying the train set into 
train and validation, with a 90/10 train/valid split

In [42]:
id_map = {}
for i, l in enumerate(train_df[1].unique()):
    id_map[l] = i

train_labels = []
for label in train_df[1]:
    train_labels.append(id_map[label])

In [43]:
id_map

{'Cell.membrane': 0,
 'Cytoplasm': 1,
 'Endoplasmic.reticulum': 2,
 'Golgi.apparatus': 3,
 'Lysosome/Vacuole': 4,
 'Mitochondrion': 5,
 'Nucleus': 6,
 'Peroxisome': 7,
 'Plastid': 8,
 'Extracellular': 9}

In [44]:
train, validation = train_test_split(train_df, test_size=0.1, stratify=train_labels)

In [45]:
print(train.shape)
print(validation.shape)

(9974, 3)
(1109, 3)


Write out FASTA files containing the sequences and subcellular location labels for the newly separated train, valid, and test splits

In [46]:
with open(f"../data/deeploc_train_{MAX_SEQ_LENGTH}.fasta", 'w') as output_train_handle:
    for index, row in train.iterrows():
        ID = row[0]
        label = row[1]
        seq = row[2]
        rec = SeqRecord(seq, id=ID, description=str(id_map[label]))
        SeqIO.write(rec, output_train_handle, "fasta")
    
with open(f"../data/deeploc_valid_{MAX_SEQ_LENGTH}.fasta", 'w') as output_valid_handle:
    for index, row in validation.iterrows():
        ID = row[0]
        label = row[1]
        seq = row[2]
        rec = SeqRecord(seq, id=ID, description=str(id_map[label]))
        SeqIO.write(rec, output_valid_handle, "fasta")

with open(f"../data/deeploc_test_{MAX_SEQ_LENGTH}.fasta", 'w') as output_test_handle:
    for index, row in test_df.iterrows():
        ID = row[0]
        label = row[1]
        seq = row[2]
        rec = SeqRecord(seq, id=ID, description=str(id_map[label]))
        SeqIO.write(rec, output_test_handle, "fasta")